In [ ]:
! pip install polars

  Using cached polars-1.24.0-cp39-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (14 kB)
Using cached polars-1.24.0-cp39-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (34.3 MB)


In [ ]:
import polars as pl

In [ ]:
from datetime import datetime, timedelta

In [ ]:
s3_path1 = 's3://ercot-60d-reports/DAM 60 day reports/dam_load_resource_data_60d/dam_load_resource_data_60d.parquet'
s3_path2 = 's3://ercot-60d-reports/DAM 60 day reports/dam_gen_resource_data_60d/dam_gen_resource_data_60d.parquet'
dam_load = pl.read_parquet(s3_path1)
dam_gen = pl.read_parquet(s3_path2)

# DAM Gen

In [ ]:
# 1. extract name prefix into a column
dam_gen = dam_gen.with_columns(
    pl.col("resource_name").str.split("_").list.get(0).alias("resource_prefix")
)

In [ ]:
dam_gen.head()

delivery_date,hour_ending,qse,dme,resource_name,resource_type,qse_submitted_curve_mw1,qse_submitted_curve_price1,qse_submitted_curve_mw2,qse_submitted_curve_price2,qse_submitted_curve_mw3,qse_submitted_curve_price3,qse_submitted_curve_mw4,qse_submitted_curve_price4,qse_submitted_curve_mw5,qse_submitted_curve_price5,qse_submitted_curve_mw6,qse_submitted_curve_price6,qse_submitted_curve_mw7,qse_submitted_curve_price7,qse_submitted_curve_mw8,qse_submitted_curve_price8,qse_submitted_curve_mw9,qse_submitted_curve_price9,qse_submitted_curve_mw10,qse_submitted_curve_price10,start_up_hot,start_up_inter,start_up_cold,min_gen_cost,hsl,lsl,resource_status,awarded_quantity,settlement_point_name,energy_settlement_point_price,regup_awarded,regup_mcpc,regdown_awarded,regdown_mcpc,rrspfr_awarded,rrsffr_awarded,rrsufr_awarded,rrs_mcpc,nonspin_awarded,nonspin_mcpc,ecrssd_awarded,ecrs_mcpc,resource_prefix
date,i32,str,str,str,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,str,f64,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,str
2023-05-02,1,"""QGOLDN""","""YGOLDE""","""AEEC_ANTLP_1""","""DSL""",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,54.0,30.0,"""OUT""",null,"""AEEC""",9.34,null,null,null,null,null,null,null,null,null,null,null,null,"""AEEC"""
2023-05-02,1,"""QGOLDN""","""YGOLDE""","""AEEC_ANTLP_2""","""DSL""",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,54.0,30.0,"""OUT""",null,"""AEEC""",9.34,null,null,null,null,null,null,null,null,null,null,null,null,"""AEEC"""
2023-05-02,1,"""QGOLDN""","""YGOLDE""","""AEEC_ANTLP_3""","""DSL""",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,54.0,30.0,"""OUT""",null,"""AEEC""",9.34,null,null,null,null,null,null,null,null,null,null,null,null,"""AEEC"""
2023-05-02,1,"""QGOLDN""","""YGOLDE""","""AEEC_ELK_1""","""SCGT90""",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,190.9,85.0,"""EMRSWGR""",null,"""AEEC""",9.34,null,null,null,null,null,null,null,null,null,null,null,null,"""AEEC"""
2023-05-02,1,"""QGOLDN""","""YGOLDE""","""AEEC_ELK_2""","""SCGT90""",85.0,19.66,96.8,20.22,108.5,20.77,120.3,21.32,132.0,21.88,143.8,22.43,155.6,22.99,167.3,23.54,179.1,24.1,190.9,24.65,5902.01,5902.01,5902.01,28.88,190.9,85.0,"""OFF""",0.0,"""AEEC""",9.34,null,null,null,null,null,null,null,null,null,null,null,null,"""AEEC"""


In [ ]:
# 2. add 'isBESS' dummy column
dam_gen = dam_gen.with_columns(
    (pl.when(pl.col("resource_type") == "PWRSTR")
     .then(1)
     .otherwise(0))
    .alias("isBESS")
)

In [ ]:
# check column values
dam_gen.filter(pl.col('resource_type') == 'PWRSTR').head()

delivery_date,hour_ending,qse,dme,resource_name,resource_type,qse_submitted_curve_mw1,qse_submitted_curve_price1,qse_submitted_curve_mw2,qse_submitted_curve_price2,qse_submitted_curve_mw3,qse_submitted_curve_price3,qse_submitted_curve_mw4,qse_submitted_curve_price4,qse_submitted_curve_mw5,qse_submitted_curve_price5,qse_submitted_curve_mw6,qse_submitted_curve_price6,qse_submitted_curve_mw7,qse_submitted_curve_price7,qse_submitted_curve_mw8,qse_submitted_curve_price8,qse_submitted_curve_mw9,qse_submitted_curve_price9,qse_submitted_curve_mw10,qse_submitted_curve_price10,start_up_hot,start_up_inter,start_up_cold,min_gen_cost,hsl,lsl,resource_status,awarded_quantity,settlement_point_name,energy_settlement_point_price,regup_awarded,regup_mcpc,regdown_awarded,regdown_mcpc,rrspfr_awarded,rrsffr_awarded,rrsufr_awarded,rrs_mcpc,nonspin_awarded,nonspin_mcpc,ecrssd_awarded,ecrs_mcpc,resource_prefix,isBESS
date,i32,str,str,str,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,str,f64,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,str,i32
2023-05-02,1,"""QBRD11""","""YBROAD""","""ALVIN_UNIT1""","""PWRSTR""",0.0,0.0,0.0,0.01,0.0,50.0,0.0,75.0,0.0,4999.0,9.9,5000.0,null,null,null,null,null,null,null,null,0.0,0.0,0.0,0.0,8.5,0.0,"""ON""",0.0,"""ALVIN_RN""",19.96,0.0,4.17,4.5,9.8,4.0,0.0,0.0,1.5,0.0,2.14,null,null,"""ALVIN""",1
2023-05-02,1,"""QENE11""","""YENELT""","""ANCHOR_BESS1""","""PWRSTR""",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,35.2,0.0,"""ONTEST""",null,"""ANCHOR_ALL""",14.45,null,null,null,null,null,null,null,null,null,null,null,null,"""ANCHOR""",1
2023-05-02,1,"""QENE11""","""YENELT""","""ANCHOR_BESS2""","""PWRSTR""",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,36.3,0.0,"""ONTEST""",null,"""ANCHOR_ALL""",14.45,null,null,null,null,null,null,null,null,null,null,null,null,"""ANCHOR""",1
2023-05-02,1,"""QBRD11""","""YBROAD""","""ANGLETON_UNIT1""","""PWRSTR""",0.0,0.0,0.0,0.01,0.0,50.0,0.0,75.0,0.0,4999.0,9.9,5000.0,null,null,null,null,null,null,null,null,0.0,0.0,0.0,0.0,6.5,0.0,"""ON""",0.0,"""BRPANGLE_RN""",14.02,0.0,4.17,4.5,9.8,2.0,0.0,0.0,1.5,0.0,2.14,null,null,"""ANGLETON""",1
2023-05-02,1,"""QENEL6""","""YENELT""","""AZURE_BESS1""","""PWRSTR""",0.0,4999.0,77.6,4999.1,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,0.0,0.0,0.0,0.0,77.6,0.0,"""OUT""",0.0,"""AZURE_RN""",14.13,null,null,null,null,null,null,null,null,null,null,null,null,"""AZURE""",1


In [ ]:
# 3. adjsut timestamp format
# Convert the 'date' column to datetime (already in YYYY-MM-DD format, so no need for a specific format)
dam_gen = dam_gen.with_columns(
    pl.col('delivery_date').cast(pl.Datetime).alias('delivery_date')
)

# Add duration
dam_gen = dam_gen.with_columns(
    (pl.col('delivery_date') + pl.duration(hours=pl.col('hour_ending'))).alias('date_time')
)

# Format the date_time column as '%m/%d/%Y %H:%M'
dam_gen = dam_gen.with_columns(
    pl.col('date_time').dt.strftime('%m/%d/%Y %H:%M').alias('date_time')
)

# Fix: Convert date_time to datetime format
dam_gen = dam_gen.with_columns(
    pl.col('date_time').str.to_datetime("%m/%d/%Y %H:%M", strict=False)
)

In [ ]:
dam_gen.head()

delivery_date,hour_ending,qse,dme,resource_name,resource_type,qse_submitted_curve_mw1,qse_submitted_curve_price1,qse_submitted_curve_mw2,qse_submitted_curve_price2,qse_submitted_curve_mw3,qse_submitted_curve_price3,qse_submitted_curve_mw4,qse_submitted_curve_price4,qse_submitted_curve_mw5,qse_submitted_curve_price5,qse_submitted_curve_mw6,qse_submitted_curve_price6,qse_submitted_curve_mw7,qse_submitted_curve_price7,qse_submitted_curve_mw8,qse_submitted_curve_price8,qse_submitted_curve_mw9,qse_submitted_curve_price9,qse_submitted_curve_mw10,qse_submitted_curve_price10,start_up_hot,start_up_inter,start_up_cold,min_gen_cost,hsl,lsl,resource_status,awarded_quantity,settlement_point_name,energy_settlement_point_price,regup_awarded,regup_mcpc,regdown_awarded,regdown_mcpc,rrspfr_awarded,rrsffr_awarded,rrsufr_awarded,rrs_mcpc,nonspin_awarded,nonspin_mcpc,ecrssd_awarded,ecrs_mcpc,resource_prefix,isBESS,date_time
datetime[μs],i32,str,str,str,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,str,f64,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,str,i32,datetime[μs]
2023-05-02 00:00:00,1,"""QGOLDN""","""YGOLDE""","""AEEC_ANTLP_1""","""DSL""",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,54.0,30.0,"""OUT""",null,"""AEEC""",9.34,null,null,null,null,null,null,null,null,null,null,null,null,"""AEEC""",0,2023-05-02 01:00:00
2023-05-02 00:00:00,1,"""QGOLDN""","""YGOLDE""","""AEEC_ANTLP_2""","""DSL""",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,54.0,30.0,"""OUT""",null,"""AEEC""",9.34,null,null,null,null,null,null,null,null,null,null,null,null,"""AEEC""",0,2023-05-02 01:00:00
2023-05-02 00:00:00,1,"""QGOLDN""","""YGOLDE""","""AEEC_ANTLP_3""","""DSL""",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,54.0,30.0,"""OUT""",null,"""AEEC""",9.34,null,null,null,null,null,null,null,null,null,null,null,null,"""AEEC""",0,2023-05-02 01:00:00
2023-05-02 00:00:00,1,"""QGOLDN""","""YGOLDE""","""AEEC_ELK_1""","""SCGT90""",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,190.9,85.0,"""EMRSWGR""",null,"""AEEC""",9.34,null,null,null,null,null,null,null,null,null,null,null,null,"""AEEC""",0,2023-05-02 01:00:00
2023-05-02 00:00:00,1,"""QGOLDN""","""YGOLDE""","""AEEC_ELK_2""","""SCGT90""",85.0,19.66,96.8,20.22,108.5,20.77,120.3,21.32,132.0,21.88,143.8,22.43,155.6,22.99,167.3,23.54,179.1,24.1,190.9,24.65,5902.01,5902.01,5902.01,28.88,190.9,85.0,"""OFF""",0.0,"""AEEC""",9.34,null,null,null,null,null,null,null,null,null,null,null,null,"""AEEC""",0,2023-05-02 01:00:00


In [ ]:
# 4. reduce columns
dam_gen = dam_gen.drop(['dme','hsl','lsl','resource_status','qse_submitted_curve_mw1',
                        'qse_submitted_curve_price1', 'qse_submitted_curve_mw2',
                        'qse_submitted_curve_price2', 'qse_submitted_curve_mw3',
                        'qse_submitted_curve_price3', 'qse_submitted_curve_mw4',
                        'qse_submitted_curve_price4', 'qse_submitted_curve_mw5',
                        'qse_submitted_curve_price5', 'qse_submitted_curve_mw6',
                        'qse_submitted_curve_price6', 'qse_submitted_curve_mw7',
                        'qse_submitted_curve_price7', 'qse_submitted_curve_mw8', 'qse_submitted_curve_price8',
                        'qse_submitted_curve_mw9', 'qse_submitted_curve_price9', 'qse_submitted_curve_mw10', 'qse_submitted_curve_price10'])

In [ ]:
dam_gen.head()

delivery_date,hour_ending,qse,resource_name,resource_type,start_up_hot,start_up_inter,start_up_cold,min_gen_cost,awarded_quantity,settlement_point_name,energy_settlement_point_price,regup_awarded,regup_mcpc,regdown_awarded,regdown_mcpc,rrspfr_awarded,rrsffr_awarded,rrsufr_awarded,rrs_mcpc,nonspin_awarded,nonspin_mcpc,ecrssd_awarded,ecrs_mcpc,resource_prefix,isBESS,date_time,resource_function,regup_revenue,regdown_revenue,rrspfr_revenue,rrsffr_revenue,rrsufr_revenue,nonspin_revenue,ecrssd_revenue,total_revenue
datetime[μs],i32,str,str,str,f64,f64,f64,f64,f64,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,str,i32,datetime[μs],str,f64,f64,f64,f64,f64,f64,f64,f64
2023-05-02 00:00:00,1,"""QGOLDN""","""AEEC_ANTLP_1""","""DSL""",null,null,null,null,null,"""AEEC""",9.34,null,null,null,null,null,null,null,null,null,null,null,null,"""AEEC""",0,2023-05-02 01:00:00,"""GEN""",null,null,null,null,null,null,null,0.0
2023-05-02 00:00:00,1,"""QGOLDN""","""AEEC_ANTLP_2""","""DSL""",null,null,null,null,null,"""AEEC""",9.34,null,null,null,null,null,null,null,null,null,null,null,null,"""AEEC""",0,2023-05-02 01:00:00,"""GEN""",null,null,null,null,null,null,null,0.0
2023-05-02 00:00:00,1,"""QGOLDN""","""AEEC_ANTLP_3""","""DSL""",null,null,null,null,null,"""AEEC""",9.34,null,null,null,null,null,null,null,null,null,null,null,null,"""AEEC""",0,2023-05-02 01:00:00,"""GEN""",null,null,null,null,null,null,null,0.0
2023-05-02 00:00:00,1,"""QGOLDN""","""AEEC_ELK_1""","""SCGT90""",null,null,null,null,null,"""AEEC""",9.34,null,null,null,null,null,null,null,null,null,null,null,null,"""AEEC""",0,2023-05-02 01:00:00,"""GEN""",null,null,null,null,null,null,null,0.0
2023-05-02 00:00:00,1,"""QGOLDN""","""AEEC_ELK_2""","""SCGT90""",5902.01,5902.01,5902.01,28.88,0.0,"""AEEC""",9.34,null,null,null,null,null,null,null,null,null,null,null,null,"""AEEC""",0,2023-05-02 01:00:00,"""GEN""",null,null,null,null,null,null,null,0.0


In [ ]:
# 5. add 'resource_function' column = 'GEN' for further concatination
dam_gen = dam_gen.with_columns(pl.lit('GEN').alias('resource_function'))

In [ ]:
# 6. add 'revenue' columns
dam_gen = dam_gen.with_columns(
    (pl.col("regup_awarded") * pl.col("regup_mcpc")).alias("regup_revenue"),
    (pl.col("regdown_awarded") * pl.col("regdown_mcpc")).alias("regdown_revenue"),
    (pl.col("rrspfr_awarded") * pl.col("rrs_mcpc")).alias("rrspfr_revenue"),
    (pl.col("rrsffr_awarded") * pl.col("rrs_mcpc")).alias("rrsffr_revenue"),
    (pl.col("rrsufr_awarded") * pl.col("rrs_mcpc")).alias("rrsufr_revenue"),
    (pl.col("nonspin_awarded") * pl.col("nonspin_mcpc")).alias("nonspin_revenue"),
    (pl.col("ecrssd_awarded") * pl.col("ecrs_mcpc")).alias("ecrssd_revenue")
)

In [ ]:
# 7. add total revenue column
dam_gen = dam_gen.with_columns(pl.sum_horizontal([
        pl.col("regup_revenue"), pl.col("regdown_revenue"),
        pl.col("rrspfr_revenue"), pl.col("rrsffr_revenue"),
        pl.col("rrsufr_revenue"), pl.col("nonspin_revenue"),
        pl.col("ecrssd_revenue")
    ]).alias("total_revenue"))

In [ ]:
# 8. export file
dam_gen.write_csv("dam_gen_v2.csv")
dam_gen.write_parquet("dam_gen_v2.parquet")

# DAM Load

In [ ]:
dam_load.head()

delivery_date,hour_ending,load_resource_name,max_power_consumption_for_load_resource,low_power_consumption_for_load_resource,regup_awarded,regup_mcpc,regdown_awarded,regdown_mcpc,rrspfr_awarded,rrsffr_awarded,rrsufr_awarded,rrs_mcpc,nonspin_awarded,nonspin_mcpc,ecrssd_awarded,ecrsmd_awarded,ecrs_mcpc
date,i32,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
2023-05-02,1,"""AAPIPLNC_LD3""",24.0,0.0,null,null,null,null,null,null,null,null,null,null,null,null,null
2023-05-02,1,"""AAPIPLNC_LD4""",15.0,0.0,null,null,null,null,null,null,null,null,null,null,null,null,null
2023-05-02,1,"""ACRLY_LD1""",2.0,0.0,null,null,null,null,null,null,null,null,null,null,null,null,null
2023-05-02,1,"""ADK_LD2""",10.0,0.0,null,null,null,null,null,null,null,null,null,null,null,null,null
2023-05-02,1,"""ADK_LD3""",10.0,0.2,null,null,null,null,null,null,null,null,null,null,null,null,null


In [ ]:
# 1. extract name prefix into a column
dam_load = dam_load.with_columns(
    pl.col("load_resource_name").str.split("_").list.get(0).alias("resource_prefix")
)

In [ ]:
# 2. add 'isBESS' dummy column by comparing to DAM Gen

# get interim bess prefix list from Gen
bess_prefixes = set(dam_gen.filter(pl.col('isBESS') == 1)['resource_prefix'].to_list())

# lookup values from the list to create column in Load
dam_load = dam_load.with_columns(
    pl.when(pl.col('resource_prefix').is_in(bess_prefixes))
    .then(1)
    .otherwise(0)
    .alias("isBESS")
)

In [ ]:
# check
dam_load.filter(pl.col('isBESS') == 1).head()

delivery_date,hour_ending,load_resource_name,max_power_consumption_for_load_resource,low_power_consumption_for_load_resource,regup_awarded,regup_mcpc,regdown_awarded,regdown_mcpc,rrspfr_awarded,rrsffr_awarded,rrsufr_awarded,rrs_mcpc,nonspin_awarded,nonspin_mcpc,ecrssd_awarded,ecrsmd_awarded,ecrs_mcpc,resource_prefix,isBESS
date,i32,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,str,i32
2023-05-02,1,"""ALVIN_LD1""",9.9,0.0,0.0,4.17,1.0,9.8,0.0,0.0,0.0,1.5,0.0,2.14,null,null,null,"""ALVIN""",1
2023-05-02,1,"""ANCHOR_LD1""",35.2,0.0,null,null,null,null,null,null,null,null,null,null,null,null,null,"""ANCHOR""",1
2023-05-02,1,"""ANCHOR_LD2""",36.3,0.0,null,null,null,null,null,null,null,null,null,null,null,null,null,"""ANCHOR""",1
2023-05-02,1,"""ANGLETON_LD1""",9.9,0.0,0.0,4.17,1.0,9.8,0.0,0.0,0.0,1.5,0.0,2.14,null,null,null,"""ANGLETON""",1
2023-05-02,1,"""AZURE_LD1""",77.6,0.0,null,null,null,null,null,null,null,null,null,null,null,null,null,"""AZURE""",1


In [ ]:
# 3. adjsut timestamp format
# Convert the 'date' column to datetime (already in YYYY-MM-DD format, so no need for a specific format)
dam_load = dam_load.with_columns(
    pl.col('delivery_date').cast(pl.Datetime).alias('delivery_date')
)

# Add duration
dam_load = dam_load.with_columns(
    (pl.col('delivery_date') + pl.duration(hours=pl.col('hour_ending'))).alias('date_time')
)

# Format the date_time column as '%m/%d/%Y %H:%M'
dam_load = dam_load.with_columns(
    pl.col('date_time').dt.strftime('%m/%d/%Y %H:%M').alias('date_time')
)

# Fix: Convert date_time to datetime format
dam_load = dam_load.with_columns(
    pl.col('date_time').str.to_datetime("%m/%d/%Y %H:%M", strict=False)
)

In [ ]:
dam_load.head()

delivery_date,hour_ending,load_resource_name,max_power_consumption_for_load_resource,low_power_consumption_for_load_resource,regup_awarded,regup_mcpc,regdown_awarded,regdown_mcpc,rrspfr_awarded,rrsffr_awarded,rrsufr_awarded,rrs_mcpc,nonspin_awarded,nonspin_mcpc,ecrssd_awarded,ecrsmd_awarded,ecrs_mcpc,resource_prefix,isBESS,date_time,resource_function,regup_revenue,regdown_revenue,rrspfr_revenue,rrsffr_revenue,rrsufr_revenue,nonspin_revenue,ecrssd_revenue,ecrsmd_revenue,total_revenue
datetime[μs],i32,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,str,i32,datetime[μs],str,f64,f64,f64,f64,f64,f64,f64,f64,f64
2023-05-02 00:00:00,1,"""AAPIPLNC_LD3""",24.0,0.0,null,null,null,null,null,null,null,null,null,null,null,null,null,"""AAPIPLNC""",0,2023-05-02 01:00:00,"""LOAD""",null,null,null,null,null,null,null,null,0.0
2023-05-02 00:00:00,1,"""AAPIPLNC_LD4""",15.0,0.0,null,null,null,null,null,null,null,null,null,null,null,null,null,"""AAPIPLNC""",0,2023-05-02 01:00:00,"""LOAD""",null,null,null,null,null,null,null,null,0.0
2023-05-02 00:00:00,1,"""ACRLY_LD1""",2.0,0.0,null,null,null,null,null,null,null,null,null,null,null,null,null,"""ACRLY""",0,2023-05-02 01:00:00,"""LOAD""",null,null,null,null,null,null,null,null,0.0
2023-05-02 00:00:00,1,"""ADK_LD2""",10.0,0.0,null,null,null,null,null,null,null,null,null,null,null,null,null,"""ADK""",0,2023-05-02 01:00:00,"""LOAD""",null,null,null,null,null,null,null,null,0.0
2023-05-02 00:00:00,1,"""ADK_LD3""",10.0,0.2,null,null,null,null,null,null,null,null,null,null,null,null,null,"""ADK""",0,2023-05-02 01:00:00,"""LOAD""",null,null,null,null,null,null,null,null,0.0


In [ ]:
# 4. add 'resource_function' column = 'GEN' for further concatination
dam_load = dam_load.with_columns(pl.lit('LOAD').alias('resource_function'))

In [ ]:
# 5. add 'revenue' columns
dam_load = dam_load.with_columns(
    (pl.col("regup_awarded") * pl.col("regup_mcpc")).alias("regup_revenue"),
    (pl.col("regdown_awarded") * pl.col("regdown_mcpc")).alias("regdown_revenue"),
    (pl.col("rrspfr_awarded") * pl.col("rrs_mcpc")).alias("rrspfr_revenue"),
    (pl.col("rrsffr_awarded") * pl.col("rrs_mcpc")).alias("rrsffr_revenue"),
    (pl.col("rrsufr_awarded") * pl.col("rrs_mcpc")).alias("rrsufr_revenue"),
    (pl.col("nonspin_awarded") * pl.col("nonspin_mcpc")).alias("nonspin_revenue"),
    (pl.col("ecrssd_awarded") * pl.col("ecrs_mcpc")).alias("ecrssd_revenue"),
    (pl.col("ecrsmd_awarded") * pl.col("ecrs_mcpc")).alias("ecrsmd_revenue")
)

In [ ]:
# add total revenue column
dam_load = dam_load.with_columns(pl.sum_horizontal([
        pl.col("regup_revenue"), pl.col("regdown_revenue"),
        pl.col("rrspfr_revenue"), pl.col("rrsffr_revenue"),
        pl.col("rrsufr_revenue"), pl.col("nonspin_revenue"),
        pl.col("ecrssd_revenue"),pl.col("ecrsmd_revenue")
    ]).alias("total_revenue"))

In [ ]:
# 5. export file
dam_load.write_csv("dam_load_v2.csv")
dam_load.write_parquet("dam_load_v2.parquet")

# Concatenating GEN & LOAD data

In [ ]:
# DAM Gen
# 1. make copy
dam_gen_2 = dam_gen

In [ ]:
dam_gen_2.head()

delivery_date,hour_ending,qse,resource_name,resource_type,start_up_hot,start_up_inter,start_up_cold,min_gen_cost,awarded_quantity,settlement_point_name,energy_settlement_point_price,regup_awarded,regup_mcpc,regdown_awarded,regdown_mcpc,rrspfr_awarded,rrsffr_awarded,rrsufr_awarded,rrs_mcpc,nonspin_awarded,nonspin_mcpc,ecrssd_awarded,ecrs_mcpc,resource_prefix,isBESS,date_time,resource_function,regup_revenue,regdown_revenue,rrspfr_revenue,rrsffr_revenue,rrsufr_revenue,nonspin_revenue,ecrssd_revenue,total_revenue
datetime[μs],i32,str,str,str,f64,f64,f64,f64,f64,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,str,i32,datetime[μs],str,f64,f64,f64,f64,f64,f64,f64,f64
2023-05-02 00:00:00,1,"""QGOLDN""","""AEEC_ANTLP_1""","""DSL""",null,null,null,null,null,"""AEEC""",9.34,null,null,null,null,null,null,null,null,null,null,null,null,"""AEEC""",0,2023-05-02 01:00:00,"""GEN""",null,null,null,null,null,null,null,0.0
2023-05-02 00:00:00,1,"""QGOLDN""","""AEEC_ANTLP_2""","""DSL""",null,null,null,null,null,"""AEEC""",9.34,null,null,null,null,null,null,null,null,null,null,null,null,"""AEEC""",0,2023-05-02 01:00:00,"""GEN""",null,null,null,null,null,null,null,0.0
2023-05-02 00:00:00,1,"""QGOLDN""","""AEEC_ANTLP_3""","""DSL""",null,null,null,null,null,"""AEEC""",9.34,null,null,null,null,null,null,null,null,null,null,null,null,"""AEEC""",0,2023-05-02 01:00:00,"""GEN""",null,null,null,null,null,null,null,0.0
2023-05-02 00:00:00,1,"""QGOLDN""","""AEEC_ELK_1""","""SCGT90""",null,null,null,null,null,"""AEEC""",9.34,null,null,null,null,null,null,null,null,null,null,null,null,"""AEEC""",0,2023-05-02 01:00:00,"""GEN""",null,null,null,null,null,null,null,0.0
2023-05-02 00:00:00,1,"""QGOLDN""","""AEEC_ELK_2""","""SCGT90""",5902.01,5902.01,5902.01,28.88,0.0,"""AEEC""",9.34,null,null,null,null,null,null,null,null,null,null,null,null,"""AEEC""",0,2023-05-02 01:00:00,"""GEN""",null,null,null,null,null,null,null,0.0


In [ ]:
# 2. add missing columns # with ld_ prefix for load
dam_gen_2 = dam_gen_2.with_columns([
    pl.lit(None).cast(pl.Float64).alias('ld_max_power_consumption_for_load_resource'),
    pl.lit(None).cast(pl.Float64).alias('ld_low_power_consumption_for_load_resource'),
    pl.lit(None).cast(pl.Float64).alias('ld_ecrsmd_awarded'),
    pl.lit(None).cast(pl.Float64).alias('ld_ecrsmd_revenue')
])

In [ ]:
# 3. ad gn_ prefix to gen-specific columns
dam_gen_2 = dam_gen_2.rename({'qse':'gn_qse',
                              'resource_type':'gn_resource_type',
                              'start_up_hot':'gn_start_up_hot',
                             'start_up_inter':'gn_start_up_inter',
                              'start_up_cold':'gn_start_up_cold',
                              'min_gen_cost':'gn_min_gen_cost',
                              'awarded_quantity':'gn_awarded_quantity',
                              'settlement_point_name':'gn_settlement_point_name',
                              'energy_settlement_point_price':'gn_energy_settlement_point_price'})

In [ ]:
# DAM Load
# 1. make copy
dam_load_2 = dam_load

In [ ]:
# 2. add missing columns # with gn_ prefix for gen
dam_load_2 = dam_load_2.with_columns([
    pl.lit(None).cast(pl.Utf8).alias('gn_qse'),
    pl.lit(None).cast(pl.Utf8).alias('gn_resource_type'),
    pl.lit(None).cast(pl.Float64).alias('gn_start_up_hot'),
    pl.lit(None).cast(pl.Float64).alias('gn_start_up_inter'),
    pl.lit(None).cast(pl.Float64).alias('gn_start_up_cold'),
    pl.lit(None).cast(pl.Float64).alias('gn_min_gen_cost'),
    pl.lit(None).cast(pl.Float64).alias('gn_awarded_quantity'),
    pl.lit(None).cast(pl.Utf8).alias('gn_settlement_point_name'),
    pl.lit(None).cast(pl.Float64).alias('gn_energy_settlement_point_price')
])

In [ ]:
# 3. rename 'resource_name' column and add ld_ to unique to load columns
dam_load_2 = dam_load_2.rename({'load_resource_name': 'resource_name',
                              'max_power_consumption_for_load_resource':'ld_max_power_consumption_for_load_resource',
                              'low_power_consumption_for_load_resource':'ld_low_power_consumption_for_load_resource',
                              'ecrsmd_awarded': 'ld_ecrsmd_awarded','ecrsmd_revenue':'ld_ecrsmd_revenue'}
                              )

In [ ]:
desired_column_order =['delivery_date','hour_ending','date_time',
 'gn_qse','resource_name','gn_resource_type','resource_prefix','isBESS','resource_function',
 'gn_start_up_hot','gn_start_up_inter','gn_start_up_cold','gn_min_gen_cost','gn_awarded_quantity','gn_settlement_point_name','gn_energy_settlement_point_price',
 'regup_awarded','regup_mcpc','regup_revenue','regdown_awarded','regdown_mcpc','regdown_revenue',
 'rrspfr_awarded','rrsffr_awarded','rrsufr_awarded','rrs_mcpc', 'rrspfr_revenue','rrsffr_revenue','rrsufr_revenue',
 'nonspin_awarded','nonspin_mcpc', 'nonspin_revenue',
 'ecrssd_awarded','ecrs_mcpc','ecrssd_revenue',
 'total_revenue',
 'ld_max_power_consumption_for_load_resource',
 'ld_low_power_consumption_for_load_resource',
 'ld_ecrsmd_awarded',
 'ld_ecrsmd_revenue']

In [ ]:
# assign order
dam_gen_2 = dam_gen_2.select(desired_column_order)

In [ ]:
dam_gen_2.head()

delivery_date,hour_ending,date_time,gn_qse,resource_name,gn_resource_type,resource_prefix,isBESS,resource_function,gn_start_up_hot,gn_start_up_inter,gn_start_up_cold,gn_min_gen_cost,gn_awarded_quantity,gn_settlement_point_name,gn_energy_settlement_point_price,regup_awarded,regup_mcpc,regup_revenue,regdown_awarded,regdown_mcpc,regdown_revenue,rrspfr_awarded,rrsffr_awarded,rrsufr_awarded,rrs_mcpc,rrspfr_revenue,rrsffr_revenue,rrsufr_revenue,nonspin_awarded,nonspin_mcpc,nonspin_revenue,ecrssd_awarded,ecrs_mcpc,ecrssd_revenue,total_revenue,ld_max_power_consumption_for_load_resource,ld_low_power_consumption_for_load_resource,ld_ecrsmd_awarded,ld_ecrsmd_revenue
datetime[μs],i32,datetime[μs],str,str,str,str,i32,str,f64,f64,f64,f64,f64,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
2023-05-02 00:00:00,1,2023-05-02 01:00:00,"""QGOLDN""","""AEEC_ANTLP_1""","""DSL""","""AEEC""",0,"""GEN""",null,null,null,null,null,"""AEEC""",9.34,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,0.0,null,null,null,null
2023-05-02 00:00:00,1,2023-05-02 01:00:00,"""QGOLDN""","""AEEC_ANTLP_2""","""DSL""","""AEEC""",0,"""GEN""",null,null,null,null,null,"""AEEC""",9.34,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,0.0,null,null,null,null
2023-05-02 00:00:00,1,2023-05-02 01:00:00,"""QGOLDN""","""AEEC_ANTLP_3""","""DSL""","""AEEC""",0,"""GEN""",null,null,null,null,null,"""AEEC""",9.34,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,0.0,null,null,null,null
2023-05-02 00:00:00,1,2023-05-02 01:00:00,"""QGOLDN""","""AEEC_ELK_1""","""SCGT90""","""AEEC""",0,"""GEN""",null,null,null,null,null,"""AEEC""",9.34,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,0.0,null,null,null,null
2023-05-02 00:00:00,1,2023-05-02 01:00:00,"""QGOLDN""","""AEEC_ELK_2""","""SCGT90""","""AEEC""",0,"""GEN""",5902.01,5902.01,5902.01,28.88,0.0,"""AEEC""",9.34,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,0.0,null,null,null,null


In [ ]:
# assign order
dam_load_2 = dam_load_2.select(desired_column_order)

In [ ]:
# merge gen and load
dam_gen_load_merged = dam_gen_2.vstack(dam_load_2)

In [ ]:
dam_gen_load_merged.tail()

delivery_date,hour_ending,date_time,gn_qse,resource_name,gn_resource_type,resource_prefix,isBESS,resource_function,gn_start_up_hot,gn_start_up_inter,gn_start_up_cold,gn_min_gen_cost,gn_awarded_quantity,gn_settlement_point_name,gn_energy_settlement_point_price,regup_awarded,regup_mcpc,regup_revenue,regdown_awarded,regdown_mcpc,regdown_revenue,rrspfr_awarded,rrsffr_awarded,rrsufr_awarded,rrs_mcpc,rrspfr_revenue,rrsffr_revenue,rrsufr_revenue,nonspin_awarded,nonspin_mcpc,nonspin_revenue,ecrssd_awarded,ecrs_mcpc,ecrssd_revenue,total_revenue,ld_max_power_consumption_for_load_resource,ld_low_power_consumption_for_load_resource,ld_ecrsmd_awarded,ld_ecrsmd_revenue
datetime[μs],i32,datetime[μs],str,str,str,str,i32,str,f64,f64,f64,f64,f64,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
2024-11-11 00:00:00,24,2024-11-12 00:00:00,null,"""WXOCF_LD3""",null,"""WXOCF""",0,"""LOAD""",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,0.0,23.0,0.0,null,null
2024-11-11 00:00:00,24,2024-11-12 00:00:00,null,"""W_HARLIN_LD1""",null,"""W""",1,"""LOAD""",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,0.0,2.0,0.0,null,null
2024-11-11 00:00:00,24,2024-11-12 00:00:00,null,"""X4_LD1""",null,"""X4""",1,"""LOAD""",null,null,null,null,null,null,null,0.0,1.61,0.0,5.0,0.49,2.45,0.0,0.0,0.0,0.81,0.0,0.0,0.0,0.0,0.99,0.0,0.0,0.81,0.0,2.45,10.0,0.0,0.0,0.0
2024-11-11 00:00:00,24,2024-11-12 00:00:00,null,"""YATES_LD1""",null,"""YATES""",0,"""LOAD""",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,0.0,5.1,0.0,null,null
2024-11-11 00:00:00,24,2024-11-12 00:00:00,null,"""ZIER_SLR_LD1""",null,"""ZIER""",1,"""LOAD""",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,0.0,40.0,0.0,null,null


In [ ]:
dam_gen_load_merged.shape

(28284999, 40)

In [ ]:
dam_gen_2.shape

(18873509, 40)

In [ ]:
dam_load_2.shape

(9411490, 40)

In [ ]:
18873509+9411490

28284999

In [ ]:
# export file
dam_gen_load_merged.write_csv("dam_gen_load_merged.csv")